<a href="https://colab.research.google.com/github/ceaumo02/Bloc_Login_Google_Futter/blob/master/02_HMM_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**bold text**## Spanish Corpus: 

* AnCora | Github: https://github.com/UniversalDependencies/UD_Spanish-AnCora

* Use conllu parser to read corpus: https://pypi.org/project/conllu/

* Universal tags POS (Documentation): https://universaldependencies.org/u/pos/

In [1]:
#@title Previus dependencies
!pip install conllu
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

Cloning into 'UD_Spanish-AnCora'...
remote: Enumerating objects: 653, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 653 (delta 58), reused 84 (delta 58), pack-reused 569
Receiving objects: 100% (653/653), 198.60 MiB | 27.55 MiB/s, done.
Resolving deltas: 100% (453/453), done.


In [ ]:
# read conllu format file and show tidily
from conllu import parse_incr
wordList = []
data_file =  open(
    '/content/UD_Spanish-AnCora/es_ancora-ud-dev.conllu', 'r', encoding='utf-8')
for tokenlist in parse_incr(data_file):
  print(tokenlist.serialize())

In [5]:
#@title Understand structure of data list
tokenlist

TokenList<Lo, cierto, es, que, a, mí, ,, me, da, un, poco, de, pena, .>

In [7]:
# join two columns
tokenlist[1]['form']+'|'+tokenlist[1]['upos']

'cierto|ADJ'

## Model train - Count Calculus:

* tags (tags) `tagCountDict`: $C(tag)$
* emissions (word|tag) `emissionProbDict`: $C(word|tag)$
* transitions (tag|prevtag) `transitionDict`: $C(tag|prevtag)$

In [ ]:
# define dictionaries
tagCountDict = {}
emissionDict = {}
transitionDict = {}

# read courpus
tagtype = 'upos'
data_file =  open(
    '/content/UD_Spanish-AnCora/es_ancora-ud-dev.conllu', 'r', encoding='utf-8')

# Read token by token
prevtag  = None
for tokenlist in parse_incr(data_file):
  for token in tokenlist:

    # count Tags (tag)
    tag = token[tagtype]
    if tag in tagCountDict.keys():
      tagCountDict[tag] += 1
    else:
      tagCountDict[tag] = 1

    # count emission Probabilities (word|tag)
    wordtag  =token['form'].lower()+'|'+token[tagtype] # (word|tag)
    if wordtag in emissionDict.keys():
      emissionDict[wordtag] += 1
    else:
      emissionDict[wordtag] = 1

    # Count transition probabilities (tag|previous_tag)
    if prevtag is None:
      prevtag = tag
      continue
    transitiontags = tag+'|'+prevtag
    if transitiontags in transitionDict.keys():
      transitionDict[transitiontags] += 1
    else:
      transitionDict[transitiontags] = 1
    prevtag = tag

transitionDict

## Model Trainig - Probabilities calculus
* Transition probabilities:
$$P(tag|prevtag) = \frac{C(prevtag, tag)}{C(prevtag)}$$

* Emission probabilities:
 $$P(word|tag) = \frac{C(word|tag)}{C(tag)}$$

In [ ]:
transitionProbDict = {} # matrix A
emissionProbDict = {} # matrix B

# validate and train model
# transition probabilities
for key in transitionDict.keys():
  tag, prevtag = key.split('|')
  if tagCountDict[prevtag]>0:
    transitionProbDict[key] = transitionDict[key]/tagCountDict[prevtag]
  else:
    print(key)

# Emission Probabilities
for key in emissionDict.keys():
  word, tag = key.split('|')
  if tagCountDict[tag]>0:
    emissionProbDict[key] = emissionDict[key]/tagCountDict[tag]
  else:
    print(key)

transitionProbDict

In [ ]:
emissionProbDict

## Save model parameters

In [15]:
import numpy as np
np.save('transitionHMM.npy', transitionProbDict)
np.save('emissionHMM.npy', emissionProbDict)